# 제 1 유형

> 데이터
* https://www.data.go.kr/data/15007122/fileData.do
* 2018년도 성인의 건강검진 데이터(흡연상태: 1-흡연, 0-비흡연)

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/DataManim/datarepo/main/smoke/train.csv")

In [ ]:
df.head()

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,구강검진수검여부,치아우식증유무,치석,혈압차
0,F,55,145,55,73.0,0.7,0.5,1.0,1.0,129.0,...,1.0,0.9,172.0,209.0,15.0,0,Y,0.0,Y,45.0
1,M,40,180,55,74.0,1.5,1.5,1.0,1.0,102.0,...,1.0,0.8,30.0,19.0,23.0,1,Y,0.0,Y,32.0
2,F,55,150,50,72.0,1.0,0.2,1.0,1.0,116.0,...,1.0,1.0,31.0,19.0,10.0,0,Y,0.0,N,44.0
3,M,40,170,85,89.0,1.0,0.8,1.0,1.0,124.0,...,1.0,1.0,26.0,38.0,108.0,1,Y,1.0,Y,44.0
4,F,40,155,45,62.0,0.5,1.0,1.0,1.0,127.0,...,1.0,0.7,20.0,11.0,13.0,0,Y,0.0,N,42.0


## Q1.
> 수축기혈압과 이완기혈압 수치의 차이를 새로운 컬럼('혈압차')으로 생성하고, 연령대 코드별 각 그룹 중 '혈압차'의 분산이 5번째로 큰 연령대 코드를 구하라.

In [ ]:
df["혈압차"] = df["수축기혈압"] - df["이완기혈압"]

In [ ]:
df.groupby("연령대코드(5세단위)", as_index=False).agg({"혈압차": pd.Series.var}).sort_values("혈압차", ascending=False).iloc[4, 0]

60

## Q2.
> 비만도를 나타내는 지표인 WHrT는 허리둘레 / 키 로 표현한다. 일반적으로 0.58 이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라.

In [ ]:
df["WHtR"] = df["허리둘레"] / df["신장(5Cm단위)"]

In [ ]:
fat_cnt = df[df["WHtR"] >= 0.58].groupby("성별코드").count()["WHtR"]
fat_cnt.loc["M"] / fat_cnt.loc["F"]

1.1693877551020408

# 제 2 유형

> 데이터
* https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction
* 자동차 보험 가입 예측
* 평가지표: f1-score

In [ ]:
import pandas as pd

train_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/test.csv")
submission_df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/submission.csv")

## 데이터 탐색

In [ ]:
train_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,188957,Female,25,1,17.0,1,< 1 Year,No,38785.0,152.0,59,0
1,275631,Male,20,1,22.0,0,< 1 Year,No,2630.0,160.0,113,0
2,329036,Female,40,1,41.0,0,1-2 Year,Yes,33165.0,124.0,188,0
3,227288,Female,28,1,30.0,1,< 1 Year,No,29038.0,152.0,233,0
4,300441,Female,50,1,28.0,0,1-2 Year,Yes,31325.0,124.0,258,0
...,...,...,...,...,...,...,...,...,...,...,...,...
304882,351529,Male,75,1,33.0,0,1-2 Year,Yes,32412.0,26.0,296,1
304883,287917,Male,57,1,28.0,1,1-2 Year,No,44602.0,52.0,135,0
304884,21023,Female,40,1,35.0,0,1-2 Year,Yes,38779.0,26.0,293,0
304885,67340,Male,24,1,41.0,0,< 1 Year,Yes,26669.0,152.0,29,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304887 entries, 0 to 304886
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    304887 non-null  int64  
 1   Gender                304887 non-null  object 
 2   Age                   304887 non-null  int64  
 3   Driving_License       304887 non-null  int64  
 4   Region_Code           304887 non-null  float64
 5   Previously_Insured    304887 non-null  int64  
 6   Vehicle_Age           304887 non-null  object 
 7   Vehicle_Damage        304887 non-null  object 
 8   Annual_Premium        304887 non-null  float64
 9   Policy_Sales_Channel  304887 non-null  float64
 10  Vintage               304887 non-null  int64  
 11  Response              304887 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 27.9+ MB


In [ ]:
train_df.describe(include="all")

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,304887.000000,304887,304887.000000,304887.000000,304887.000000,304887.000000,304887,304887,304887.000000,304887.000000,304887.000000,304887.000000
unique,NaN,2,NaN,NaN,NaN,NaN,3,2,NaN,NaN,NaN,NaN
top,NaN,Male,NaN,NaN,NaN,NaN,1-2 Year,Yes,NaN,NaN,NaN,NaN
freq,NaN,164764,NaN,NaN,NaN,NaN,160409,153869,NaN,NaN,NaN,NaN
mean,190603.266174,NaN,38.828304,0.997898,26.382650,0.458258,NaN,NaN,30576.579598,112.044954,154.319466,0.122563
std,110023.298778,NaN,15.504898,0.045804,13.236626,0.498255,NaN,NaN,17265.678980,54.198329,83.625500,0.327936
min,1.000000,NaN,20.000000,0.000000,0.000000,0.000000,NaN,NaN,2630.000000,1.000000,10.000000,0.000000
25%,95239.500000,NaN,25.000000,1.000000,15.000000,0.000000,NaN,NaN,24414.500000,29.000000,82.000000,0.000000
50%,190666.000000,NaN,36.000000,1.000000,28.000000,0.000000,NaN,NaN,31663.000000,133.000000,154.000000,0.000000
75%,285948.500000,NaN,49.000000,1.000000,35.000000,1.000000,NaN,NaN,39410.000000,152.000000,227.000000,0.000000


## 전처리 함수 작성

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

def my_preprocess(org_df, target="train", encode_list=None):
    df = org_df.copy()
    df.drop("id", axis=1, inplace=True)

    if target == "train":
        encoder1 = LabelEncoder()
        encoder2 = LabelEncoder()
        scaler1 = StandardScaler()
        df["Gender"] = encoder1.fit_transform(df["Gender"])
        df["Vehicle_Damage"] = encoder2.fit_transform(df["Vehicle_Damage"])
        df = pd.get_dummies(df, columns=["Vehicle_Age"])
        scaled_df = scaler1.fit_transform(df)
        df = pd.DataFrame(scaled_df, columns=df.columns)

        return df, [encoder1, encoder2, scaler1]
    
    if target == "test":
        encoder1 = encode_list[0]
        encoder2 = encode_list[1]
        scaler1 = encode_list[2]
        df["Gender"] = encoder1.transform(df["Gender"])
        df["Vehicle_Damage"] = encoder2.transform(df["Vehicle_Damage"])
        df = pd.get_dummies(df, columns=["Vehicle_Age"])
        scaled_df = scaler1.transform(df)
        df = pd.DataFrame(scaled_df, columns=df.columns)
        return df

## 불균형 데이터

In [ ]:
train_df["Response"].sum() / train_df["Response"].count()

0.12256344153735646

## 훈련, 검증 데이터 분류 및 전처리

In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop("Response", axis=1)
y = train_df["Response"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=5)

In [ ]:
X_train_proc, encode_ls = my_preprocess(X_train, target="train", encode_list=None)
X_val_proc = my_preprocess(X_val, target="test", encode_list=encode_ls)
test_proc = my_preprocess(test_df, target="test", encode_list=encode_ls)

In [ ]:
print(X_train_proc.shape)
print(X_val_proc.shape)
print(test_proc.shape)

(243909, 12)
(60978, 12)
(76222, 12)


## 오버샘플링

In [ ]:
from imblearn.over_sampling import SMOTE

smote_sample = SMOTE(random_state=5)
X_train_smote, y_train_smote = smote_sample.fit_resample(X_train_proc, y_train)

X_train_smote.shape

(428030, 12)

## 분석

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score, confusion_matrix
f1_model = []

### 로지스틱 회귀

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train_smote, y_train_smote)
pred = model_lr.predict(X_val_proc)

f1_model.append(f1_score(y_val, pred))
f1_model[0]

0.39691482960450747

In [ ]:
confusion_matrix(y_val, pred)

array([[31672, 21832],
       [  218,  7256]])

### 랜덤 포레스트

In [ ]:
scores = []
for i in range(10, 16):
    model_rf = RandomForestClassifier(max_depth=i, random_state=5)
    model_rf.fit(X_train_smote, y_train_smote)
    pred = model_rf.predict(X_val_proc)
    scores.append(f1_score(y_val, pred))

pd.DataFrame(scores, index=range(10, 16), columns=["f1"]).sort_values("f1", ascending=False)

,f1
15,0.439442
14,0.437277
13,0.434645
12,0.432032
11,0.431054
10,0.427815


In [ ]:
model_rf = RandomForestClassifier(max_depth=15, random_state=5)
model_rf.fit(X_train_smote, y_train_smote)
pred = model_rf.predict(X_val_proc)

f1_model.append(f1_score(y_val, pred))
f1_model[1]

0.43944221742556616

### AdaBoost

In [ ]:
model_ab = AdaBoostClassifier(random_state=5)
model_ab.fit(X_train_smote, y_train_smote)
pred = model_ab.predict(X_val_proc)

f1_model.append(f1_score(y_val, pred))
f1_model[2]

0.43086733034029545

### GradientBoosting

In [ ]:
model_gb = GradientBoostingClassifier(max_depth=6, random_state=5)
model_gb.fit(X_train_smote, y_train_smote)
pred = model_gb.predict(X_val_proc)

f1_model.append(f1_score(y_val, pred))
f1_model[3]

0.44634643842738875

### KNN

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=35)
model_knn.fit(X_train_smote, y_train_smote)
pred = model_knn.predict(X_val_proc)

f1_model.append(f1_score(y_val, pred))
f1_model[4]

0.4172833151019883

### ANN(인공신경망)

In [ ]:
import numpy as np
import tensorflow as tf

X_train_feature = X_train_smote.copy()
X_train_feature = np.array(X_train_feature)
X_val_feature = X_val_proc.copy()
X_val_feature = np.array(X_val_feature)

X_train_feature

array([[ 0.92100897,  1.62582366,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836],
       [ 0.92100897,  1.43229971,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836],
       [ 0.92100897, -0.05138391,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836],
       ...,
       [ 0.92100897,  1.40775454,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836],
       [ 0.92100897,  0.44864814,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836],
       [-1.08576576,  0.78721987,  0.04608845, ...,  0.95106639,
        -0.87401553, -0.20901836]])

In [ ]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(64, activation="relu"))
ann.add(tf.keras.layers.Dense(1, activation="sigmoid"))

ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.binary_accuracy])

In [ ]:
ann.fit(X_train_feature, y_train_smote, batch_size=32, epochs=50, validation_split=0.2, callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10))

Epoch 1/50
10701/10701 [==============================] - 26s 2ms/step - loss: 0.4270 - binary_accuracy: 0.7696 - val_loss: 0.5273 - val_binary_accuracy: 0.8467
Epoch 2/50
10701/10701 [==============================] - 24s 2ms/step - loss: 0.4218 - binary_accuracy: 0.7733 - val_loss: 0.5321 - val_binary_accuracy: 0.8306
Epoch 3/50
10701/10701 [==============================] - 23s 2ms/step - loss: 0.4207 - binary_accuracy: 0.7737 - val_loss: 0.5001 - val_binary_accuracy: 0.8566
Epoch 4/50
10701/10701 [==============================] - 25s 2ms/step - loss: 0.4199 - binary_accuracy: 0.7741 - val_loss: 0.5258 - val_binary_accuracy: 0.8466
Epoch 5/50
10701/10701 [==============================] - 24s 2ms/step - loss: 0.4192 - binary_accuracy: 0.7742 - val_loss: 0.5013 - val_binary_accuracy: 0.8565
Epoch 6/50
10701/10701 [==============================] - 25s 2ms/step - loss: 0.4187 - binary_accuracy: 0.7746 - val_loss: 0.5410 - val_binary_accuracy: 0.8351
Epoch 7/50
10701/10701 [==========

In [ ]:
pred = ann.predict(X_val_feature)

1906/1906 [==============================] - 5s 2ms/step


In [ ]:
for i in range(len(pred)):
    if pred[i] > 0.5:
        pred[i] = 1
    else:
        pred[i] = 0

In [ ]:
print(f1_score(y_val, pred))
print(confusion_matrix(y_val, pred))

0.44787114947094725
[[39488 14016]
 [ 1273  6201]]


In [ ]:
f1_model.append(f1_score(y_val, pred))

In [ ]:
pd.DataFrame(f1_model, index=["LogisticReg", "RandomForest", "AdaBoost", "GradientBoosting", "KNN", "ANN"])

,0
LogisticReg,0.396915
RandomForest,0.439442
AdaBoost,0.430867
GradientBoosting,0.446346
KNN,0.417283
ANN,0.447871


### ANN 의 성능이 가장 좋음

In [ ]:
test_feature = np.array(test_proc)

test_pred = ann.predict(test_feature)

2382/2382 [==============================] - 4s 2ms/step


In [ ]:
for i in range(len(test_pred)):
    if test_pred[i] > 0.5:
        test_pred[i] = 1
    else:
        test_pred[i] = 0
test_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
submission_df["0"] = test_pred
submission_df.to_csv("submission_pred.csv")